<H1>LOAD MAIL AND PUT INTO EXCEL</H1>


In [ ]:
from load_mail import main

main()

<H1>COMBINE ALL EXCEL</H1>

In [1]:
import openpyxl
import os
import pandas as pd
from datetime import date, datetime, timedelta

############### DAYS #############
# if monday, set 3 = friday, 2 = saturday, 1 = sunday
daystime = 1


EXCEL_FILE_PATH = r"D:\Workstuff\my-work-python-script\download_mail_to_folder\xlsx"
yesterday = date.today() - timedelta(days=daystime)

In [2]:
#Create excel
os.chdir(EXCEL_FILE_PATH)
new_wb = openpyxl.Workbook()
new_wb.save(f"Parcel Pickup {yesterday}.xlsx")
load_new_wb = openpyxl.load_workbook(os.path.join(EXCEL_FILE_PATH, f"Parcel Pickup {yesterday}.xlsx"), data_only=True)
sheet = load_new_wb.get_sheet_by_name('Sheet')
sheet.cell(row=1,column=1).value = "Tracking"
sheet.cell(row=1,column=2).value = "Time"
sheet.title = f"{yesterday}"
load_new_wb.save(filename=f"Parcel Pickup {yesterday}.xlsx")


C:\Users\Comseven\AppData\Local\Temp\ipykernel_23436\2831042389.py:6: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  sheet = load_new_wb.get_sheet_by_name('Sheet')


In [3]:
#Combine all into one
for file in os.listdir(EXCEL_FILE_PATH):
    data = pd.read_excel(file, dtype=object) ##turn excel into dataframe
    with pd.ExcelWriter(f"Parcel Pickup {yesterday}.xlsx", mode="a", if_sheet_exists="overlay", engine="openpyxl") as excel_file:
        excel_file.number_format = openpyxl.styles.numbers.FORMAT_TEXT
        data.to_excel(excel_file, sheet_name=f"{yesterday}", header=False, index=False, startrow=excel_file.sheets[f"{yesterday}"].max_row)


c:\Users\Comseven\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Comseven\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Comseven\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Comseven\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, app

In [4]:
#Load file and convert time to 24h format
load_time = pd.read_excel(f"Parcel Pickup {yesterday}.xlsx")
shipment_number = pd.DataFrame(load_time['Tracking'])

time_column_shipment = pd.DataFrame(load_time['Time'].str.split("([\d][\d][:][\d][\d] [A-Z][A-Z])", expand=True, regex=True))
frames = [shipment_number, time_column_shipment]
new_data = pd.concat(frames, axis=1)
with pd.ExcelWriter(f"Parcel Pickup {yesterday}.xlsx", mode="a", if_sheet_exists="replace", engine="openpyxl") as excel_file:
    new_data.to_excel(excel_file,sheet_name=f"{yesterday}", header=False, index=False, startrow=1)

#time_column = pd.DataFrame(excel_file['Time'].str.split(" ", expand = True))

<h1>LOAD ALL DATA FROM STOCKOUT</h1>

In [28]:
#stockout

stockout = pd.read_clipboard(sep='\t')
stockoutid = stockout["Stock Out (ID)"]
branchid = stockout["Branch (ID)"]
booking_id = stockout["Booking ID"].str.rstrip().str.replace("Booking-DHL ID : ","").str.split(" , ", expand=True)

frames = [stockoutid, branchid, booking_id]
data = pd.concat(frames, axis=1)

C:\Users\Comseven\AppData\Local\Temp\ipykernel_11568\2680818527.py:3: DtypeWarning: Columns (11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  stockout = pd.read_clipboard(sep='\t')


In [27]:
#stockout_insure

stockout_insure = pd.read_clipboard(sep='\t')
stockoutid_insure = stockout_insure["Stock Out (ID)"]
branchid_insure = stockout_insure["Branch (ID)"] #
booking_id_insure = stockout_insure["Booking ID"].str.rstrip().str.replace("Booking-DHL ID : ","").str.split(" , ", expand=True)

frames_insure = [stockoutid_insure, branchid_insure, booking_id_insure]
data_insure = pd.concat(frames_insure, axis=1)

<h1>INSERT DATAFRAME TO EXCEL THEN INDEX/MATCH</h1>

In [29]:
with pd.ExcelWriter(f"Parcel Pickup {yesterday}.xlsx", mode="a", if_sheet_exists="overlay", engine="openpyxl") as excel_file:
    data.to_excel(excel_file, sheet_name="stockout")
    data_insure.to_excel(excel_file, sheet_name="stockout_insure")

In [11]:
#time convert algorithm

def convert24(str1):
    #check if am and equal 12
    if str1[-2:] == "AM" and str1[:2] == "12":
        return "00" + str1[2:-2]

    #otherwise just remove AM
    elif str1[-2:] == "AM":
        return str1[:-2]
    
    #but if it's pm and equal 12
    elif str1[-2:] == "PM" and str1[:2] == "12":
        return str1[:-2]

    else:
        return str(int(str1[:2]) + 12) + str1[2:5]
    

In [30]:
load_wb = openpyxl.load_workbook(os.path.join(EXCEL_FILE_PATH, f"Parcel Pickup {yesterday}.xlsx"), data_only=True)
load_sheet = load_wb.sheetnames
shipment_data = load_wb[load_sheet[0]]
stockout_normal = load_wb[load_sheet[1]]
stockout_insurance = load_wb[load_sheet[2]]

for row in range(2,shipment_data.max_row+1):
    shipment_data.cell(row=row, column=3).value = convert24(shipment_data.cell(row=row, column=3).value)
    shipment_data.cell(row=row, column=4).value = f'''=IF(ISNUMBER(MATCH({openpyxl.utils.quote_sheetname(shipment_data.title)}!A{row},stockout!D:D,0)),INDEX(stockout!B:B,MATCH({openpyxl.utils.quote_sheetname(shipment_data.title)}!A{row},stockout!D:D,0)),IF(ISNUMBER(MATCH({openpyxl.utils.quote_sheetname(shipment_data.title)}!A{row},stockout_insure!D:D,0)),INDEX(stockout_insure!B:B,MATCH({openpyxl.utils.quote_sheetname(shipment_data.title)}!A{row},stockout_insure!D:D,0)),""))'''
    shipment_data.cell(row=row, column=5).value = f'''=IF(ISNUMBER(MATCH({openpyxl.utils.quote_sheetname(shipment_data.title)}!A{row},stockout!D:D,0)),INDEX(stockout!C:C,MATCH({openpyxl.utils.quote_sheetname(shipment_data.title)}!A{row},stockout!D:D,0)),IF(ISNUMBER(MATCH({openpyxl.utils.quote_sheetname(shipment_data.title)}!A{row},stockout_insure!D:D,0)),INDEX(stockout_insure!C:C,MATCH({openpyxl.utils.quote_sheetname(shipment_data.title)}!A{row},stockout_insure!D:D,0)),""))'''
    shipment_data.cell(row=row, column=6).value = f'''=IF(ISNUMBER(MATCH({openpyxl.utils.quote_sheetname(shipment_data.title)}!A{row},stockout!D:D,0)),"Stockout",IF(ISNUMBER(MATCH({openpyxl.utils.quote_sheetname(shipment_data.title)}!A{row},stockout_insure!D:D,0)),"Stockout_Insure",""))'''
load_wb.save(f"Parcel Pickup {yesterday}.xlsx")